<a href="https://colab.research.google.com/github/ubiodee/SingularityNet/blob/main/RAG_Github.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install necessary libraries
!pip install transformers langchain chromadb PyGithub sentence-transformers
!pip install GitPython
!pip install -U langchain-community


from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.chains import RetrievalQA
from langchain.document_loaders import GitLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.2/599.2 kB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 88.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 362.6/362.6 kB 31.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.3 MB/s eta 

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:132: UserWarning: Field "model_name" in HuggingFaceInferenceAPIEmbeddings has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [ ]:
# Clean the text by removing unwanted syntax
import re

def clean_text(text):
    # Remove any markdown, HTML, or GitHub embed syntax
    text = re.sub(r'{%.*?%}', '', text)  # Remove GitHub embeds
    text = re.sub(r'\[.*?\]\(.*?\)', '', text)  # Remove markdown links
    text = re.sub(r'<.*?>', '', text)  # Remove HTML tags
    return text.strip()

In [ ]:
# Load files from the timeline folder
def load_and_clean_files_from_folder(folder_path):
    documents = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".md"):  # Load markdown files
                with open(os.path.join(root, file), "r", encoding="utf-8") as f:
                    content = f.read()
                    cleaned_content = clean_text(content)  # Clean the text before processing
                    documents.append(cleaned_content)
    return documents

In [ ]:
repo_url = "https://github.com/SingularityNET-Archive/SingularityNET-Archive-GitBook.git"
repo_dir = "/content/singularitynet_archive"
import os
from git import Repo
if not os.path.exists(repo_dir):
    Repo.clone_from(repo_url, repo_dir)
timeline_folder = os.path.join(repo_dir, "timeline")

# Load files from the timeline folder
def load_and_clean_files_from_folder(folder_path):
    documents = []
    for root, _, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".md"):  # Load markdown files
                with open(os.path.join(root, file), "r", encoding="utf-8") as f:
                    content = f.read()
                    cleaned_content = clean_text(content)  # Clean the text before processing
                    documents.append(cleaned_content)
    return documents

In [ ]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

<ipython-input-5-83794808db26>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or d

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from langchain.schema import Document # import Document object from langchain.schema

# The function is defined as 'load_and_clean_files_from_folder'
documents = load_and_clean_files_from_folder(timeline_folder)
# Convert the list of strings to a list of Document objects
documents = [Document(page_content=doc) for doc in documents]

In [ ]:
vectorstore = Chroma.from_documents(documents, embeddings)

In [ ]:
model_name = "NousResearch/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/177 [00:00<?, ?B/s]

In [ ]:
# Create a pipeline with the model
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, max_new_tokens=200, temperature= 0.7)

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
!pip install -U langchain-huggingface
import langchain_huggingface
!pip install HuggingFacePipeline

ERROR: Could not find a version that satisfies the requirement HuggingFacePipeline (from versions: none)
ERROR: No matching distribution found for HuggingFacePipeline


In [ ]:
llm = HuggingFacePipeline(pipeline=pipe)

<ipython-input-15-b97a330b5f24>:1: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [ ]:
# Wrap the pipeline in a HuggingFacePipeline object for use with LangChain
llm = HuggingFacePipeline(pipeline=pipe)

In [ ]:
!pip install langchain
from langchain import PromptTemplate

# Create a PromptTemplate to guide the model's output
prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    Use the following context to answer the question in a clear and concise manner. Do not include any unnecessary details, markdown syntax, or HTML embeds.
    Also do not add the introductory query to the response you are giving and dont state your internal promtings just go straight to answering the question.

    Context: {context}

    Question: {question}
    """,
)

In [ ]:
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())


In [ ]:
# Example query
query = "What is the timeline of events in the SingularityNET Archive?"
response = qa_chain.invoke({"query": query})

# Process the response to remove excess '\n' characters and make it more readable
def clean_response(response):
    cleaned_response = response['result'].replace("\\n", "\n").strip()  # Replace \n with actual new lines and remove extra spaces
    return cleaned_response

# Clean the output
formatted_response = clean_response(response)
print(formatted_response)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

---
description: The events of May 2022 related to SingularityNET
---

# May 2022

## Summary

The first Ambassador Program Town hall, the "kick-off", was held on Tuesday, 10th May.

## Twitter Search

SingularityNET until:2022-06-01 since:2022-05-01

---
description: The SingularityNET Ambassador events of April 2023
---

# April 2023

## Overview

In April 2023 the SingularityNET Decentralisation Summit was held and Felix and Tevo Saks from the Ambassador programme participated.

## GitHub Issue for this month

---
description: 4th September to 10th September 2023
---

# Week 37



## Monday 4th September 2023 

### Education Guild

No summary given

***

## Tuesday 5th September 2023

### Governance WorkGroup

Not this week

***

### Ambassador Town Hall

Town Hall #64.\
No summary given.

#### Video



***

## Wednesday 